In [1]:
!pip3 install sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///data.sqlite')
#connect to the database
con = engine.connect()
# cursor  



  


  Using cached https://files.pythonhosted.org/packages/7f/84/263ba21a9f2725a1b7236823bd732d80e23a4d79b2b121060b3ce039fd7b/SQLAlchemy-1.3.18-cp36-cp36m-manylinux1_x86_64.whl


In [2]:
### Q1: Some of the facilities charge a fee to members, but some do not.
##Please list the names of the facilities that do.###


In [3]:
sql_cmd = """
SELECT name, membercost
FROM Facilities
WHERE (SELECT CAST(membercost AS float)) > 0.0
GROUP BY name
LIMIT 0 , 30 """
rs = con.execute(sql_cmd)
print (rs.fetchall())


[('Massage Room 1', '9.9'), ('Massage Room 2', '9.9'), ('Squash Court', '3.5'), ('Tennis Court 1', '5.0'), ('Tennis Court 2', '5.0')]


In [4]:
###/* Q2: How many facilities do not charge a fee to members? */
sql_cmd1 = """
SELECT COUNT(name)
FROM Facilities
WHERE (SELECT CAST(membercost AS float)) = 0.0"""
rs = con.execute(sql_cmd1)
print (rs.fetchall())


[(4,)]


In [5]:
#/* Q3: How can you produce a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost?
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. */

sql_cmd2 = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE (SELECT CAST(membercost as float)) < (SELECT CAST(monthlymaintenance as float)) * 0.2 """
rs = con.execute(sql_cmd2)
print (rs.fetchall())


[('0', 'Tennis Court 1', '5.0', '200'), ('1', 'Tennis Court 2', '5.0', '200'), ('2', 'Badminton Court', '0.0', '50'), ('3', 'Table Tennis', '0.0', '10'), ('4', 'Massage Room 1', '9.9', '3000'), ('5', 'Massage Room 2', '9.9', '3000'), ('6', 'Squash Court', '3.5', '80'), ('7', 'Snooker Table', '0.0', '15'), ('8', 'Pool Table', '0.0', '15')]


In [6]:
#/* Q4: How can you retrieve the details of facilities with ID 1 and 5?
#Write the query without using the OR operator. */
sql_cmd3 = """
SELECT *
FROM Facilities
WHERE (SELECT CAST(facid as int)) IN (1, 5)
"""
rs = con.execute(sql_cmd3)
print (rs.fetchall())


[('1', 'Tennis Court 2', '5.0', '25.0', '8000', '200'), ('5', 'Massage Room 2', '9.9', '80.0', '4000', '3000')]


In [7]:
#/* Q5: How can you produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100? Return the name and monthly maintenance of the facilities
#in question. */
sql_cmd4 = """
SELECT name, monthlymaintenance,
  CASE WHEN (SELECT CAST(monthlymaintenance as float)) > 100.0 THEN 'expensive'
  ELSE 'cheap' END AS maintenance_cheap_or_expensive
FROM Facilities"""
rs = con.execute(sql_cmd4)
print (rs.fetchall())

[('Tennis Court 1', '200', 'expensive'), ('Tennis Court 2', '200', 'expensive'), ('Badminton Court', '50', 'cheap'), ('Table Tennis', '10', 'cheap'), ('Massage Room 1', '3000', 'expensive'), ('Massage Room 2', '3000', 'expensive'), ('Squash Court', '80', 'cheap'), ('Snooker Table', '15', 'cheap'), ('Pool Table', '15', 'cheap')]


In [8]:
#/* Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Do not use the LIMIT clause for your solution. */

sql_cmd5= """SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members)"""
rs = con.execute(sql_cmd5)
print (rs.fetchall())

[('Darren', 'Smith', '2012-09-26 18:08:45')]


In [9]:
#/* Q7: How can you produce a list of all members who have used a tennis court?
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. */
sql_cmd6="""SELECT Facilities.name, (Members.firstname || Members.surname) AS member_name
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Facilities.name LIKE 'Tennis Court%'
GROUP BY Facilities.name, Members.firstname, Members.surname
ORDER BY Members.firstname"""
rs = con.execute(sql_cmd6)
print (rs.fetchall())

[('Tennis Court 1', 'AnneBaker'), ('Tennis Court 2', 'AnneBaker'), ('Tennis Court 1', 'BurtonTracy'), ('Tennis Court 2', 'BurtonTracy'), ('Tennis Court 1', 'CharlesOwen'), ('Tennis Court 2', 'CharlesOwen'), ('Tennis Court 2', 'DarrenSmith'), ('Tennis Court 1', 'DavidFarrell'), ('Tennis Court 1', 'DavidJones'), ('Tennis Court 1', 'DavidPinker'), ('Tennis Court 2', 'DavidFarrell'), ('Tennis Court 2', 'DavidJones'), ('Tennis Court 1', 'DouglasJones'), ('Tennis Court 1', 'EricaCrumpet'), ('Tennis Court 1', 'FlorenceBader'), ('Tennis Court 2', 'FlorenceBader'), ('Tennis Court 1', 'GUESTGUEST'), ('Tennis Court 2', 'GUESTGUEST'), ('Tennis Court 1', 'GeraldButters'), ('Tennis Court 2', 'GeraldButters'), ('Tennis Court 2', 'HenriettaRumney'), ('Tennis Court 1', 'JackSmith'), ('Tennis Court 2', 'JackSmith'), ('Tennis Court 1', 'JaniceJoplette'), ('Tennis Court 2', 'JaniceJoplette'), ('Tennis Court 1', 'JemimaFarrell'), ('Tennis Court 2', 'JemimaFarrell'), ('Tennis Court 1', 'JoanCoplin'), ('Tenn

In [10]:

#/* Q8: How can you produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30? Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries. */
sql_cmd7="""SELECT Facilities.name,
  (Members.firstname || Members.surname) AS member_name,
  CASE WHEN (SELECT CAST(Members.memid as int)) !=0
    THEN (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.membercost as float))
    ELSE (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.guestcost as float))
    END AS booking_cost
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Bookings.starttime LIKE '2012-09-14%'
  AND(
    (SELECT CAST(Bookings.memid as float)) != 0 AND (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.membercost as float)) > 30.0
    OR (SELECT CAST(Bookings.memid as float)) = 0 AND (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.guestcost as float)) > 30.0
  )
ORDER BY booking_cost DESC"""
rs = con.execute(sql_cmd7)
print (rs.fetchall())

[('Massage Room 2', 'GUESTGUEST', 320.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Tennis Court 2', 'GUESTGUEST', 150.0), ('Tennis Court 1', 'GUESTGUEST', 75.0), ('Tennis Court 1', 'GUESTGUEST', 75.0), ('Tennis Court 2', 'GUESTGUEST', 75.0), ('Squash Court', 'GUESTGUEST', 70.0), ('Massage Room 1', 'JemimaFarrell', 39.6), ('Squash Court', 'GUESTGUEST', 35.0), ('Squash Court', 'GUESTGUEST', 35.0)]


In [11]:

#/* Q9: This time, produce the same result as in Q8, but using a subquery. */

sql_cmd8="""SELECT *
FROM (
  SELECT Facilities.name,
  (Members.firstname || Members.surname) AS member_name,
  CASE WHEN (SELECT CAST(Members.memid as int)) !=0
    THEN (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.membercost as float))
    ELSE (SELECT CAST(Bookings.slots as float)) * (SELECT CAST (Facilities.guestcost as float))
    END AS booking_cost
  FROM Bookings
  INNER JOIN Members ON Bookings.memid = Members.memid
  INNER JOIN Facilities ON Bookings.facid = Facilities.facid
  WHERE Bookings.starttime LIKE '2012-09-14%'
  ) sub
WHERE (SELECT CAST(sub.booking_cost as float)) > 30.0
ORDER BY sub.booking_cost DESC"""
rs = con.execute(sql_cmd8)
print (rs.fetchall())

[('Massage Room 2', 'GUESTGUEST', 320.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Massage Room 1', 'GUESTGUEST', 160.0), ('Tennis Court 2', 'GUESTGUEST', 150.0), ('Tennis Court 1', 'GUESTGUEST', 75.0), ('Tennis Court 1', 'GUESTGUEST', 75.0), ('Tennis Court 2', 'GUESTGUEST', 75.0), ('Squash Court', 'GUESTGUEST', 70.0), ('Massage Room 1', 'JemimaFarrell', 39.6), ('Squash Court', 'GUESTGUEST', 35.0), ('Squash Court', 'GUESTGUEST', 35.0)]


In [12]:
#/* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */

sql_cmd9="""SELECT facility, total_revenue
FROM (
  SELECT Facilities.name AS facility,
  SUM(
    CASE WHEN (SELECT CAST(Bookings.memid as int)) !=0
    THEN (SELECT CAST(Bookings.slots as float)) * (SELECT CAST (Facilities.membercost as float))
    ELSE (SELECT CAST(Bookings.slots as float)) * (SELECT CAST(Facilities.guestcost as float)) END) AS total_revenue
  FROM Facilities
  INNER JOIN Bookings ON Bookings.facid = Facilities.facid
  GROUP BY Facilities.name
) AS facility_total_revenue
WHERE total_revenue < 1000
ORDER BY total_revenue"""

rs = con.execute(sql_cmd9)
print (rs.fetchall())

[('Table Tennis', 180.0), ('Snooker Table', 240.0), ('Pool Table', 270.0)]
